In [4]:
# !wget https://raw.githubusercontent.com/Vladimir-Dimitrov-Ngu/NLP_course/hw-2/hw_2/data%20spam/spam_or_not_spam.csv

--2023-10-05 08:22:20--  https://raw.githubusercontent.com/Vladimir-Dimitrov-Ngu/NLP_course/hw-2/hw_2/data%20spam/spam_or_not_spam.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4256112 (4.1M) [text/plain]
Saving to: ‘spam_or_not_spam.csv.3’

spam_or_not_spam.cs 100%[===================>]   4.06M  --.-KB/s    in 0.09s   

2023-10-05 08:22:20 (43.7 MB/s) - ‘spam_or_not_spam.csv.3’ saved [4256112/4256112]


In [10]:
import pandas as pd
import spacy
import fasttext
from tqdm import trange
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

RANDOM_STATE = 123

ModuleNotFoundError: No module named 'pandas'

In [8]:
import fasttext

ModuleNotFoundError: No module named 'fasttext'

In [7]:
df = pd.read_csv('spam_or_not_spam.csv')

In [8]:
df.sample(1)

,email,label
2674,check out our NUMBER NUMBER NUMBER internet c...,1


In [9]:
df.dropna(inplace=True)

In [10]:
nlp = spacy.load("en_core_web_sm")
stopwords = nlp.Defaults.stop_words

In [11]:
df["cleaned_email"] = df["email"].apply(
    lambda x: " ".join(
        token.lemma_.lower()
        for token in nlp(x)
        if not token.is_stop
        and not token.is_punct
        and not token.is_digit
        and not token.like_email
        and not token.like_num
        and not token.is_space
    )
)
df.drop(columns='email', inplace=True)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    df["cleaned_email"], df["label"], random_state=RANDOM_STATE, test_size=0.2,
    stratify = df['label'],
    shuffle=True
    )

### FastText

In [13]:
for index in range(X_train.shape[0]):
  X_train.iloc[index] = X_train.iloc[index] + ' __label__' + str(y_train.iloc[index])

In [14]:
with open("train_email.txt", "w") as f:
    for email in X_train:
        f.write("%s\n" % email)
del X_train

In [ ]:
model = fasttext.train_supervised('train_email.txt', dim=1500, thread=4, wordNgrams=2,
                            ws=5)
# fasttext.train_unsupervised('clean_email.txt', dim=300, thread=4, wordNgrams=3,
#                             model='cbow', ws=5)
# model.save_model("fasttextmodel.bin")

In [ ]:
predicts = []
for index in trange(X_test.shape[0]):
  predict_label = model.predict(X_test.iloc[index], 1)[0][0]
  if predict_label == '__label__0':
    predicts.append(0)
  else:
    predicts.append(1)

In [ ]:
print(f1_score(y_test, predicts))
print(classification_report(y_test, predicts, zero_division=0))